# 异步、事件驱动的IO

> 当今高性能服务器底层采用的主流IO方式
>
> 这里不再讨论基于进程或线程模型的IO


单线程的事件循环 Event Loop

- 等待事件
- 循环处理事件队列中的所有事件

示例：Nginx的事件循环

![](event-loop.png)

每个事件的处理：调用注册的回调函数

如果一个回调阻塞，则整个事件循环所在的线程被阻塞

![](blocking-operation.png)

一种解决办法：异步回调（比如Node.js)
- 将任何可能阻塞的调用都改成非阻塞调用加回调的方式
- 容易导致**callback hell**

另一种解决办法：**线程池**

- 将回调的阻塞部分作为一个任务提交给线程池
- 线程池执行完成任务后，以事件的方式通知事件循环
- 事件循环执行回调的剩余部分

> 将阻塞任务“卸载”给任务队列和线程池，
>
> **阻塞式请求**本身仍然被阻塞，
>
> 但其它所有**非阻塞式请求**不再被阻塞

![](thread-pools-worker-process-event-cycle.png)

即使使用线程池，能并发处理的**阻塞式请求**的数量还是有限的，等于线程池的大小

在这种模型或框架下，业务处理能做的优化：
- 预估或规定阻塞式请求数量的上限，相应地设置线程池大小
- 将阻塞式任务拆分为非阻塞式，比如读文件时，将同步阻塞调用改为，同步非阻塞调用（把fd交给事件循环）

依然带来心智负担：
- 线程池需要设置最小和最大线程数，以及队列最大长度
- 任务拆分，需要在拆分后的多个回调间保存状态，还要加上错误处理和超时机制

> 处理大并发的阻塞任务简直是噩梦！

更优雅的解决方法：执行体不再是线程，而是**轻量级线程/协程/绿色线程**

编程视角：
- 同步IO：降低编程难度
- 尽可能多的执行体：提供大并发IO

要求：
- 更少的资源占用
- 接近于函数调用的上下文切换成本

轻量级线程/协程/绿色线程在很多实现中底层依然使用了事件循环

> 漂亮优雅地封装了脏活累活